爬取新乡市的历史天气数据，并且用在模型中，实例代码：https://yonniye.com/archives/11.html     2345里面新乡的网页：https://tianqi.2345.com/wea_history/53986.htm

In [ ]:
import requests
import urllib.request
from tqdm import tqdm
from keys import *

In [ ]:
months = []
year = 2020
for i in range(12):
    months.append("%d%02d"%(year, i+1))
year = 2021
for i in range(12):
    months.append("%d%02d"%(year, i+1))
year = 2022
for i in range(10):
    months.append("%d%02d"%(year, i+1))
selected_month = months[22]


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
     Chrome/77.0.3865.120 Safari/537.36"
}


todo_url = f"http://tianqi.2345.com/t/wea_history/js/{selected_month}/58238_{selected_month}.js"

In [ ]:
def get_history_temps(months):
    """
        这个函数可以按照月份需求读取2345天气网上的历史数据，并搜索得到历史上某一天的最高温和最低温
        months输入为列表，格式为%yyyy%mm
        原始方法：https://yonniye.com/archives/11.html
    """
    date_list = []
    temp_high_list = []
    temp_low_list = []
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
     Chrome/77.0.3865.120 Safari/537.36"}
    for selected_month in months:
        todo_url = f"http://tianqi.2345.com/t/wea_history/js/{selected_month}/58238_{selected_month}.js"
        """ 
            这里默认读出来的数据是乱码，实例如下：
            2021-11-01',bWendu:'23Ўж',yWendu:'14Ўж',tianqi:'¶аФЖ',fengxiang:'¶«ұұ·з',fengli:'2ј¶',aqi:'74',aqiInfo:'Бј',aqiLevel:'2'},{
        """     
        data = requests.get(todo_url,headers=headers)
        print(data.text)
        data = data.text.lstrip("var weather_str=").rstrip(';')
        month_list = data.split('ymd:\'')[1:-1]
        for line in month_list:
            date = line[:10]
            if 'Ўж' in data:
                temp_high = int(line[10:].split('Ўж')[0].split(':\'')[1])
                temp_low = int(line[10:].split('Ўж')[1].split(':\'')[1])
            elif 'Ёц' in data:
                temp_high = int(line[10:].split('Ёц')[0].split(':\'')[1])
                temp_low = int(line[10:].split('Ёц')[1].split(':\'')[1])  

            date_list.append(date)
            temp_high_list.append(temp_high)
            temp_low_list.append(temp_low)
    return date_list,temp_high_list,temp_low_list
        


In [ ]:
data

In [ ]:
date_list,temp_high_list,temp_low_list = get_history_temps(months=months[:1])


In [ ]:
import pandas


temp_data = {
    'date': date_list,
    'max_temp': temp_high_list,
    'min_temp': temp_low_list
    }
df = pandas.DataFrame(temp_data)



In [ ]:
df

In [ ]:
df['date'] = df['date'].apply(lambda x: x.replace('-',''))
df['date'] = pandas.to_datetime(df['date'])

In [ ]:
df.to_csv(
    "Histor_temperature_{}_{}.csv".format(
        months[0],months[-1]
    ),
    encoding = 'utf-8-sig'

)